# Understand embeddings with Word2Vec

### Exercise objectives:
- Convert words to vector representations thanks to embeddings
- Discover the powerful Word2Vec algorithm

<hr>
<hr>

_Embeddings_ are representation of words thanks to vectors. These embeddings can be learnt within a Neural Network. But it can take time to converge. Another option is to learn them as a first step. Then, use them directly to feed the word representation into an RNN. 



# The data

Keras provides many datasets, among which the ÌMDB dataset: it corresponds to sentences that are movie reviews. Each of them is related to a score given by the review writer.

❓ **Question** ❓ Let's first load the data. You don't have to understand what is going on in the function, it does not matter here.

⚠️ **Warning** ⚠️ The `load_data` function has a `percentage_of_sentences` argument. Depending on your computer, there are chances that a too large number of sentences will make your compute slow down, or even freeze - your RAM can even overflow. For that reason, **you should start with 10% of the sentences** and see if your computer handles it. Otherwise, rerun with a lower number. 

⚠️ **DISCLAIMER** ⚠️ **No need to play _who has the biggest_ (RAM) !** The idea is to get to run your models quickly to prototype. Even in real life, it is recommended that you start with a subset of your data to loop and debug quickly. So increase the number only if you are into getting the best accuracy. 

In [1]:
###########################################
### Just run this cell to load the data ###
###########################################

import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)
    
    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)
        
        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]
  
        len_test = int(percentage_of_sentences/100*len(test_sentences))
        test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]
    
    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]
    
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=10)

In the previous exercise, we jointly learnt a representation for the words, and feed this representation to a RNN, as shown here : 

<img src="layers_embedding.png" width="400px" />

However, this increases the number of parameters to learn, which can slow the convergence, and make it harder!

For that reason, we will separate the steps of learning the word representation and feeding it to a RNN. As shown here : 

<img src="word2vec_representation.png" width="400px" />

We will learn the embedding with the word2vec.

The drawback is indeed that the learnt embedding is not _specifically_ designed for our task. However, learning it independently of the task at hand (sentiment analysis) has some advantages : 
- it is very fast to do in general (with word2vec)
- the representation learnt by word2vec is still meaningful 
- the convergence of the RNN alone will be easier and faster

So let's learn an embedding with word2vec and see how meaningful it is!


# Embedding with Word2Vec

Let's use Word2Vec to embed the words of our sentences. Word2Vec will be able to convert each word to a fixed-size vectorial representation.

For instance, we will have:
- 'dog' --> [0.1, -0.3, 0.8]
- 'cat' --> [-1.1, 2.3, 0.7]
- 'apple' --> [3.1, 0.9, -4.7]

Here, your embedding space is of size 3.

What you expect is to have representation such as words with close meanings are close in this embedding space. As on this example:

![Embedding](word_embedding.png)

❓ **Question** ❓ Let's run Word2Vec! The following code imports word2vec from [GENSIM](https://radimrehurek.com/gensim/), a great python package that makes the use of the word2vec algorithm easy, fast and accurate - which is not an easy task. The second line learns the embedding representation of the words thanks to the sentences in `X_train`.

In [2]:
from gensim.models import Word2Vec

word2vec = Word2Vec(sentences=X_train)

Let's look at the embedded representation of some words.

You can use `word2vec.wv` as a dictionary.
For instance, `word2vec.wv['dog']` will return a representation of `dog` in the embedding space.

❓ **Question** ❓ Try different words - especially, try non-existing words to see that they don't have any representation (which is perfectly normal as their representation were not learn). 

In [3]:
word2vec.wv['cat']

array([-0.2147675 ,  0.19513148, -0.26038754,  0.37116334, -0.05178672,
       -0.25770366,  0.14611736,  0.71860933, -0.2041894 , -0.30314022,
       -0.02732908, -0.40912184,  0.01440527,  0.15576671, -0.10793411,
       -0.2627964 ,  0.25965375, -0.14290181, -0.0170959 , -0.45975265,
        0.136135  ,  0.0441258 ,  0.2765331 , -0.18798096, -0.04912954,
        0.12827206, -0.33867937, -0.08040594, -0.31292686,  0.07356417,
        0.14129987,  0.00191541,  0.13226652, -0.59110504, -0.32457253,
        0.22415178,  0.17136341, -0.1599468 , -0.3652608 , -0.2233498 ,
       -0.13005596, -0.38460147, -0.51088625,  0.23671368,  0.40172505,
       -0.07543164, -0.17193538, -0.11090108,  0.46492878,  0.27975604,
        0.12661415, -0.38546762, -0.1894139 , -0.01517787, -0.33145827,
        0.21617588,  0.24155276, -0.14914232, -0.11186227,  0.07615358,
        0.26080808, -0.07280325,  0.07117338,  0.14364992, -0.16059804,
        0.27090314, -0.04431881,  0.35130954, -0.47618553,  0.30

❓ **Question** ❓ What is the size of each word representation, and therefore, what is the size of the embedding space?

In [4]:
word2vec.wv.vector_size  # ===> 100

100

How to know if this embedding make any sense? To do that, we will check that words with a close meaning have close representations. 

Let's use the `word2vec.most_similar(...)` method that, given an input word, display the "closest" words in the embedding space. If the embedding is well done, then words that have close meanings will have close representation in the embedding space.

❓ **Question** ❓ Test the `most_similar` method on different words. 

❗ **Remark** ❗ Indeed, the quality of the closeness will depend on the quality of your embedding, and thus, depend on the number of sentences that you have loaded and from which you create your embedding.

In [5]:
word2vec.wv.most_similar('book')

[('series', 0.9448404312133789),
 ('version', 0.9144883751869202),
 ('week', 0.8995492458343506),
 ('case', 0.8992325663566589),
 ('word', 0.8890294432640076),
 ('episode', 0.8887625336647034),
 ('complaint', 0.8882715106010437),
 ('opinion', 0.8871198892593384),
 ('trailer', 0.8839464783668518),
 ('adaptation', 0.8837878108024597)]

Similarly to `most_similar` used on words directly, we can use `similar_by_vector` on vectors to do the same thing :

In [6]:
word2vec.wv.similar_by_vector('review')

[('comment', 0.9649448394775391),
 ('crap', 0.9532013535499573),
 ('expectations', 0.9461610913276672),
 ('regret', 0.9439267516136169),
 ('rated', 0.9344543814659119),
 ('miserably', 0.9335386753082275),
 ('rate', 0.9324734210968018),
 ('missed', 0.930023729801178),
 ('option', 0.9281923174858093),
 ('purchase', 0.9270244240760803)]

# Arithmetic on words

Now, let's do mathematical operations on words - meaning on their vector representations!

As any word is represented as a vector, you can do basic arithmetic as:

$$W2V(good) - W2V(bad)$$

❓ **Question** ❓ Do this mathematical operation and print the result

In [7]:
word2vec.wv['earth'] - word2vec.wv['sky']

array([-0.15815918, -0.14233635,  0.01103693,  0.02329417, -0.0205646 ,
       -0.04457772,  0.04072095,  0.1832816 , -0.02377616, -0.02085432,
       -0.23444009, -0.15399033, -0.12963875,  0.04404323, -0.09827545,
       -0.06746408,  0.19419645, -0.08194928,  0.02945624, -0.08086041,
        0.08614236,  0.11054747, -0.09480189,  0.074468  ,  0.09045537,
        0.05633505, -0.18731488,  0.04720185, -0.06972231,  0.14957535,
        0.07022768,  0.15058124,  0.05160452, -0.22859299, -0.1690724 ,
        0.15585022, -0.00368324, -0.01384453, -0.04935253, -0.05369031,
        0.02816182, -0.00408471, -0.09462368,  0.06402475,  0.05344534,
       -0.0961832 ,  0.02839062,  0.05313888,  0.16653982,  0.05748054,
        0.15286194, -0.13430938,  0.19945441,  0.03467166, -0.13771233,
        0.12576717,  0.11026029, -0.08516662, -0.10448632,  0.06216914,
        0.10134049,  0.1707452 , -0.04138826,  0.00708914, -0.18564363,
       -0.00861266,  0.1557846 , -0.00997218, -0.11864847,  0.20

Now, image for a second that, somehow, the following equality holds true - just for a second

$$W2V(good) - W2V(bad) = W2V(nice) - W2V(stupid)$$

which is equivalent to 

$$W2V(good) - W2V(bad) + W2V(stupid) = W2V(nice)$$

❓ **Question** ❓ Let's, just for fun (as it would be foolish of us to think that this equality holds true ...), do the operation $W2V(good) - W2V(bad) + W2V(stupid)$ and store it in a `res` variable (which will be a vector of size 100 that you can print).

In [8]:
res = word2vec.wv['good'] - word2vec.wv['bad'] + word2vec.wv['stupid']
res

array([-0.1611327 , -0.34296483,  0.40375805,  0.01098278,  0.4068828 ,
       -0.9190555 , -0.44069093,  0.4329912 , -0.5357651 , -0.62945646,
       -0.09180571, -0.45457903,  0.00625887,  0.21837887,  0.22113807,
        0.35510874, -0.06585926, -0.4006841 , -0.6045048 , -0.69048554,
        0.12883258,  0.29438555,  0.43639755, -0.04941738,  0.06420669,
       -0.18207407, -0.4343784 ,  0.50896174, -0.3561704 , -0.58337265,
        0.43302196,  0.43293536,  0.49108312, -0.09647426, -0.43430725,
        0.24848509,  0.25680268, -0.18167078, -0.5472373 , -0.11746997,
       -0.22169988, -0.6077771 , -0.30836546,  0.0802601 ,  0.7461068 ,
       -0.12464391,  0.2872916 , -0.54838115, -0.23916055,  0.8674896 ,
        0.08207449, -0.2212493 , -0.69816107,  0.36590433,  0.18883169,
        0.51378226,  0.1573362 ,  0.4503427 , -0.24736089,  0.5242189 ,
        0.1329908 , -0.05922261,  0.19994068, -0.16294208, -0.50889313,
        0.38240358, -0.331415  , -0.19060796,  0.38518083,  0.73

We earlier said that, for any vector, it is possible to see the closest vectors in the embedding space.

❓ **Question** ❓ Look at the closest vector (thanks to the `word2vec.wv.similar_by_vector` function) of `res`

In [9]:
word2vec.wv.similar_by_vector(res)

[('good', 0.7352895140647888),
 ('nice', 0.7141351699829102),
 ('alexandra', 0.7065533399581909),
 ('always', 0.7062939405441284),
 ('great', 0.7031354308128357),
 ('given', 0.7020660638809204),
 ('potential', 0.6984413862228394),
 ('considered', 0.6912691593170166),
 ('revolver', 0.6876935958862305),
 ('cannes', 0.678972601890564)]

Incredible right! You can do arithmetic operations on words!

❓ **Question** ❓ You can try on arithmetic such as 

$$W2V(Boy) - W2V(Girl) = W2V(Man) - W2V(Woman)$$

or 

$$W2V(Queen) - W2V(King) = W2V(actress) - W2V(actor)$$

❗ **Remark** ❗ You will probably see that the results are not perfect. But don't forget that you trained your model on a very small corpus.

In [10]:
word2vec.wv['boy'] - word2vec.wv['girl'] == word2vec.wv['man'] - word2vec.wv['woman']

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False])

In [11]:
word2vec.wv['queen'] - word2vec.wv['king'] == word2vec.wv['actress'] - word2vec.wv['actor']

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False])

You might wonder where does this magic comes from (at quite a low price, you just run a line of code on a very small corpus and it was trained within few minutes). The magic comes from the way Word2Vec is trained. The details are quite complex, but you can remember that Word2vec, in `word2vec = Word2Vec(sentences=X_train)` , actually trains a internal neural network (that you don't see).  

In a nutshell, this internal NN predicts a word from the surroundings words in a sentences. So it chooses many splits in the different sentences, choose some words as inputs $X$  and a word as output $y$ which it tries to predict, in the embedding space.

And as any neural network, Word2Vec has some hyperparameters. Let's check some. 


# Word2Vec hyperparameters


❓ **Question** ❓ The first important hyperparameter is the `vector_size` argument. It corresponds to the size of the embedding space. Learn a new `word2vec_2` model, still trained on the `X_train`, but with a smaller or higher `vector_size`.

Verify on some words that the corresponding embedding is of your selected size.

In [12]:
word2vec_2 = Word2Vec(X_train, vector_size=78)

In [13]:
word2vec_2.wv['car'], len(word2vec_2.wv['car'])

(array([ 0.04455859,  0.02740472, -0.2886299 , -0.09125005,  0.73672384,
        -0.30307972, -0.21625878, -0.15771487,  0.13665208, -0.32628083,
         0.16581707, -0.1535944 ,  0.09449906,  0.2646753 , -0.11170214,
         0.26375237, -0.25473124, -0.34175953,  0.2505808 , -0.35785723,
        -0.11574816,  0.08949723,  0.03054591, -0.1265429 ,  0.07082261,
        -0.46313903,  0.162431  ,  0.08128066,  0.42693835, -0.09039369,
        -0.30737516, -0.73449105, -0.12219489, -0.25053045,  0.17276593,
         0.26220158,  0.17475183,  0.20668827,  0.35206437,  0.13444267,
         0.3381542 ,  0.7031131 , -0.42618302, -0.17832713,  0.23611075,
        -0.00712764, -0.4941848 , -0.24013013,  0.11336972, -0.5358997 ,
        -0.33032492, -0.27645862, -0.40603086,  0.44698343,  0.14929664,
         0.09431393, -0.22883323,  0.543923  ,  0.35850385, -0.3541718 ,
         0.05270565,  0.9733864 , -0.6773655 ,  0.20280264, -0.33717367,
         0.08219059,  0.2990731 , -0.38938138, -0.1

❓ **Question** ❓ Use the `word2vec.wv.key_to_index` attribute to display the size of the learnt vocabulary. On the other hand, compare it to the number of different words in `X_train`.

In [14]:
word2vec.wv.key_to_index['car'], len(X_train)

(635, 2500)

In [15]:
X_train;

There is an important difference between the number of words in the train sentences and in the Word2Vec vocabulary, even though it has been train on the train sentence set. The reasons comes from the second important hyperparameter of Word2Vec :  `min_count`. 

`min_count` is a integer that tells you how many occurences a given word should have to be learn in the embedding space. For instance, let's say that the word "movie" appears 1000 times in the corpus and "simba" only 2 times. If `min_count=3`, the word "simba" will be skipped during the training.

The intention is to have only words that are sufficiently present in the corpus to have a robust embedded representation

❓ **Question** ❓ Learn a new `word2vec_3` model with a `min_count` higher than 5 (which is the default value) and a `word2vec_4` with a `min_count` smaller than 5, and then, compare the size of the vocabulary for all the different word2vec that you have trained (you can choose any `vector_size` you want).

In [16]:
word2vec_3 = Word2Vec(X_train, min_count=7)
word2vec_4 = Word2Vec(X_train, min_count=4)

In [17]:
len(word2vec_2.wv.key_to_index), len(word2vec_3.wv.key_to_index), len(word2vec_4.wv.key_to_index)

(8006, 6064, 9584)

Remember that we say that word2vec has an internal neural network that it optimizes based on some predictions? These predictions actually correspond to predicting a word based on surrounding words. The surroundings words are in a `window` which corresponds to the number of words taken into account. And you can train the word2vec with different `window` sizes.

❓ **Question** ❓ Learn a new `word2vec_5` model with a `window` different than previously (default is 5).

In [18]:
word2vec_5 = Word2Vec(X_train, window=12)

The arguments you have seen (`vector_size`, `min_count` and `window`) are usually the one that you should start changing to get a better performance for your model.

But you can also look at other arguments in the [documentation](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Text8Corpus)



# Convert our train and test set to RNN ready data

Remember that word2vec is the first step to the overall process of feeding such a representation into a RNN, as shown here :

<img src="word2vec_representation.png" width="400px" />



Now, let's work on Step 2 by converting the training and test data into their vector representation to be ready to be feed in RNNs.

❓ **Question** ❓ Now, write a function that, given a sentence, returns a matrix that corresponds to the embedding of the full sentence, which means that you have to embed each word one after the other and concatenate the result to output a 2D matrix (be sure that your output is a NumPy array)

❗ **Remark** ❗ You will probably notice that some words you are trying to convert throw errors as they are said not to belong to the dictionary:

- for the test set, this is understandable: some words were not in the train set and thus their embedded representation is unknown
- for the train set, due to `min_count` hyperparameter, not all the words have a vector representation

In any case, just skip the missing words here.

In [19]:
import numpy as np

example = ['this', 'movie', 'is', 'the', 'worst', 'action', 'movie', 'ever']
example_missing_words = ['this', 'movie', 'is', 'laaaaaaaaaame']

def embed_sentence(word2vec, sentence):
    return np.array([word2vec.wv[word] for word in sentence if word in word2vec.wv])


### Checks
embedded_sentence = embed_sentence(word2vec, example)
assert(type(embedded_sentence) == np.ndarray)
assert(embedded_sentence.shape == (8, 100))

embedded_sentence_missing_words = embed_sentence(word2vec, example_missing_words)  
assert(type(embedded_sentence_missing_words) == np.ndarray)
assert(embedded_sentence_missing_words.shape == (3, 100))

❓ **Question** ❓ Write a function that, given a list of sentence (each sentence being a list of words/strings), returns a list of embedded sentences (each sentence is a matrix). Apply this function to the train and test sentences

Hint: Use the previous function `embed_sentence`

In [20]:
def embedding(word2vec, sentences):
    return [list(embed_sentence(word2vec, sentence)) for sentence in sentences]
    
X_train = embedding(word2vec, X_train)
X_test = embedding(word2vec, X_test)

❓ **Question** ❓ In order to have ready-to-use data, do not forget to pad them in order to have tensors that can be divided in batch sizes during the optimization. Store the padedd values in `X_train_pad` and `X_test_pad`. Do not forget the important arguments of the padding ;)

In [21]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [22]:
X_train_pad = pad_sequences(X_train, dtype='float32', padding='post', value=0)
X_test_pad = pad_sequences(X_test, dtype='float32', padding='post', value=0)

In [23]:
assert(len(X_train_pad.shape) == 3)
assert(len(X_test_pad.shape) == 3)
assert(X_train_pad.shape[2] == 100)
assert(X_test_pad.shape[2] == 100)